<a href="https://colab.research.google.com/github/Farhana-Najnin/fastapi-resume-matcher/blob/main/Copy_of_fastapi_resume_matcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%writefile .env
NGROK_AUTH_TOKEN=37wLo9kxLkDmJmNAA9gIoogMsIU_2SA3ynwHC2QHjWZNEov66


Writing .env


In [5]:
from dotenv import load_dotenv
import os

load_dotenv()  # loads .env automatically

NGROK_TOKEN = os.getenv("NGROK_AUTH_TOKEN")
print(NGROK_TOKEN)

37wLo9kxLkDmJmNAA9gIoogMsIU_2SA3ynwHC2QHjWZNEov66


In [6]:
!pip install -q fastapi uvicorn nest_asyncio pyngrok sentence-transformers scikit-learn


In [7]:
import re
import nest_asyncio
import numpy as np
from fastapi import FastAPI
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pyngrok import ngrok
import uvicorn
from dotenv import load_dotenv
import os

# ----------------
# Load .env and ngrok token
# ----------------
load_dotenv()  # automatically loads .env file
NGROK_TOKEN = os.getenv("NGROK_AUTH_TOKEN")
if not NGROK_TOKEN:
    raise RuntimeError("NGROK_AUTH_TOKEN not found! Check your .env file.")
nest_asyncio.apply()
app = FastAPI(title="Ultra Fast Resume Matcher")
model = SentenceTransformer("all-MiniLM-L6-v2", device="cpu")
EMBED_DIM = model.get_sentence_embedding_dimension()


SKILLS = {
    "python","machine","learning","deep","nlp",
    "sql","fastapi","docker","aws","azure","gcp",
    "pandas","numpy","scikit","tensorflow",
    "pytorch","git","linux","data","analysis"
}


class MatchRequest(BaseModel):
    resume: str
    job_description: str


def clean(text: str) -> str:
    # limit text size to avoid overload
    return re.sub(r"\s+", " ", text.lower()).strip()[:3000]

def extract_skills(text: str):
    words = set(re.findall(r"\w+", text.lower()))
    return list(SKILLS.intersection(words))

@app.post("/match")
async def match(req: MatchRequest):

    resume_text = clean(req.resume)
    job_text = clean(req.job_description)


    vectors = model.encode(
        [resume_text, job_text],
        convert_to_numpy=True
    )

    score = cosine_similarity(
        [vectors[0]], [vectors[1]]
    )[0][0] * 100

    resume_skills = extract_skills(req.resume)
    job_skills = extract_skills(req.job_description)

    return {
        "match_score_percent": round(float(score), 2),
        "resume_skills": resume_skills,
        "job_required_skills": job_skills,
        "missing_skills": list(set(job_skills) - set(resume_skills))
    }


ngrok.kill()
ngrok.set_auth_token(NGROK_TOKEN)
public_url = ngrok.connect(8000).public_url
print("Swagger Docs:", public_url + "/docs")

config = uvicorn.Config(
    app,
    host="0.0.0.0",
    port=8000,
    log_level="error"
)

server = uvicorn.Server(config)

await server.serve()



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Swagger Docs: https://endurable-thwartedly-somer.ngrok-free.dev/docs
